In [1]:
import os
import pickle
import itertools
import pandas as pd
import numpy as np
#from modelgym import model
import functools
import modelgym
from modelgym.util import TASK_CLASSIFICATION
from modelgym.trainer import Trainer
from modelgym.tracker import ProgressTrackerFile, ProgressTrackerMongo
from sklearn.metrics import roc_auc_score
from hyperopt.mongoexp import MongoTrials
from modelgym.util import split_and_preprocess
from sklearn.model_selection import train_test_split
from collections import OrderedDict

In [2]:
########### NROWS, N_ESTIMATORS, N_PROBES, TEST_SIZE, N_CV_SPLITS, OPTIMIZER
config_tuple = {
    'test': (1000, 100,  2, 0.5, 2, 'random'),
    'pror': (None, 1000, 100, 0.5, 2, 'random'), # production with random hyperopt suggestor
    'prot': (None, 1000, 100, 0.5, 2, 'tpe'),    # production with tpe hyperopt suggestor
    'demi': (10000, 100, 5, 0.5, 2, 'random')
}
CONFIG = 'test' if 'EXP_CONFIG' not in os.environ else os.environ['EXP_CONFIG']
NROWS, N_ESTIMATORS, N_PROBES, TEST_SIZE, N_CV_SPLITS, OPTIMIZER = config_tuple[CONFIG]
CANDIDATES = OrderedDict([
    ('XGBoost', modelgym.XGBModel), 
    ('LightGBM', modelgym.LGBModel),
    ('RandomForestClassifier',modelgym.RFModel)
])
RESULTS_DIR = "results"
LOAD_CACHE = False
if 'MONGO_PORT_27017_TCP_ADDR' in os.environ:
    mongo_host = os.environ['MONGO_PORT_27017_TCP_ADDR'] if 'MONGO_PORT_27017_TCP_ADDR' in os.environ else 'cern-mc01h'
    mongo_port = int(os.environ['MONGO_PORT_27017_TCP_PORT']) if 'MONGO_PORT_27017_TCP_PORT' in os.environ else 27017
    mongo_db = os.environ['MONGO_DB'] if 'MONGO_DB' in os.environ else 'trials'
    tracker_factory = functools.partial(ProgressTrackerMongo, mongo_host, mongo_port, mongo_db, config_key=CONFIG)
    print ("Using Mongo as backend for tracking")
else:
    tracker_factory = functools.partial(ProgressTrackerFile, RESULTS_DIR, config_key=CONFIG)
    print ("Using File as backend for tracking")

print ("Running experiment cofiguration:", CONFIG)

Using File as backend for tracking
Running experiment cofiguration: test


## Download & read data file

In [3]:
%%bash 
if [ ! -d data ] ; then 
    mkdir data 
    cd data
    curl https://cernbox.cern.ch/index.php/s/N1dpSAPgl30szYM/download | gunzip -c > XY2d.pickle
    cd ..
fi
ls -l data

total 266224
-rw-r--r--  1 macbook  staff  136304022 Aug 11 14:40 XY2d.pickle


In [4]:
def read_data(fname, nrows=None, shuffle=True):
    with open(fname,'rb') as fh:
        X, y = pickle.load(fh,encoding='bytes')
    index = np.arange(X.shape[0])
    if nrows is None:
        nrows = X.shape[0]
    weights = np.ones(nrows) # uh, well...
    if shuffle:
        index_perm = np.random.permutation(index)
    else:
        index_perm = index
    return X[index_perm[:nrows]], y[index_perm[:nrows]], weights


X, y, weights = read_data("data/XY2d.pickle", nrows=NROWS)

In [5]:
X_train, X_test, y_train, y_test, w_train, w_test = train_test_split(X, y, weights, test_size=TEST_SIZE)

In [6]:
cv_pairs, (dtrain, dtest) = split_and_preprocess(X_train.copy(), y_train, 
                                                X_test.copy(), y_test, 
                                                cat_cols=[], n_splits=N_CV_SPLITS)

## Run them all

In [7]:
trackers = {}
def init_keys_dict():
    return dict([(k, None) for k in CANDIDATES.keys()])
default_cv_result = init_keys_dict()
tuned_cv_result = init_keys_dict()
default_test_result = init_keys_dict()
tuned_test_result = init_keys_dict()
trials = init_keys_dict()
trainer = Trainer(hyperopt_evals=N_PROBES, n_estimators=N_ESTIMATORS)

In [8]:
for model_id, model_class in CANDIDATES.items():
    model = model_class(TASK_CLASSIFICATION)
    print ("~"*20, model.get_name(), "~"*20)
    trackers[model_id] = tracker_factory(model_name=model.get_name())
    if LOAD_CACHE:
        default_cv_result[model_id], default_test_result[model_id], tuned_cv_result[model_id], tuned_test_result[model_id], trials[model_id] = \
            trackers[model_id].load_state(as_list=True)
    
    
    if default_cv_result[model_id] is None:
        default_cv_result[model_id] = trainer.crossval_fit_eval(model, cv_pairs)
        trackers[model_id].save_state(default_cv=default_cv_result[model_id])
    trainer.print_result(default_cv_result[model_id], 'Default {} result on CV'.format(model.get_name()))

    if default_test_result[model_id] is None:
        default_test_result[model_id] = trainer.fit_eval(model, dtrain, dtest,
                                                  default_cv_result[model_id]['params'],
                                                  default_cv_result[model_id]['best_n_estimators'],
                                                  custom_metric = {'roc_auc': roc_auc_score})
        trackers[model_id].save_state(default_test=default_test_result[model_id])

    trainer.print_result(default_test_result[model_id], 'Default {} result on TEST'.format(model.get_name()), extra_keys=['roc_auc'])

        
    if tuned_cv_result[model_id] is None:
        print('Hyperopt iterations:\n\n')
        tuned_cv_result[model_id] = trainer.crossval_optimize_params(model, cv_pairs,  algo_name=OPTIMIZER, 
                                                           trials=trials[model_id], tracker=trackers[model_id])
        trackers[model_id].save_state(tuned_cv=tuned_cv_result[model_id])
    trainer.print_result(tuned_cv_result[model_id], 'Tuned {} result on cv'.format(model.get_name()))

    if tuned_test_result[model_id] is None:
        tuned_test_result[model_id] = trainer.fit_eval(model, dtrain, dtest,
                                            tuned_cv_result[model_id]['params'],
                                            tuned_cv_result[model_id]['best_n_estimators'],
                                            custom_metric = {'roc_auc': roc_auc_score})
        trackers[model_id].save_state(tuned_test=tuned_test_result[model_id])
    trainer.print_result(tuned_test_result[model_id], 'Tuned {} result on test'.format(model.get_name()), extra_keys=['roc_auc'])

    trackers[model_id].save_state(default_cv=default_cv_result[model_id], default_test=default_test_result[model_id], 
                               tuned_cv=tuned_cv_result[model_id], tuned_test=tuned_test_result[model_id], trials=trials[model_id])
    

~~~~~~~~~~~~~~~~~~~~ XGBoost ~~~~~~~~~~~~~~~~~~~~
BST  <xgboost.core.Booster object at 0x11ba5e860>
RES [0.64834, 0.613588, 0.579124, 0.553156, 0.531911, 0.51178, 0.498368, 0.48626, 0.471671, 0.460346, 0.451913, 0.4434, 0.434178, 0.427874, 0.422223, 0.416232, 0.410336, 0.409307, 0.40522, 0.403873, 0.39925, 0.395655, 0.395225, 0.392728, 0.390809, 0.39105, 0.389073, 0.388983, 0.387494, 0.387394, 0.386772, 0.386518, 0.386605, 0.390619, 0.391758, 0.393327, 0.392117, 0.392921, 0.394343, 0.398404, 0.396895, 0.398934, 0.399838, 0.398764, 0.402347, 0.403971, 0.404857, 0.404412, 0.405809, 0.40757, 0.408082, 0.405352, 0.406318, 0.406161, 0.407693, 0.410377, 0.408491, 0.41, 0.408896, 0.411079, 0.410914, 0.412946, 0.412625, 0.41353, 0.415453, 0.416017, 0.417886, 0.418556, 0.419715, 0.420142, 0.420288, 0.419522, 0.421604, 0.423237, 0.425535, 0.42517, 0.426509, 0.4267, 0.428924, 0.43041, 0.430087, 0.430419, 0.432133, 0.433322, 0.434188, 0.435031, 0.435541, 0.438144, 0.437668, 0.437895, 0.439896, 0.4

BST  <xgboost.core.Booster object at 0x11ba5e828>
RES [0.585305, 0.509317, 0.469549, 0.435704, 0.412622, 0.398995, 0.389503, 0.384602, 0.381498]
saved state to results/tracker_test_XGBoost.pickle
Tuned XGBoost result on test:

loss = 0.381498
n_estimators = 9
params = {'alpha': 0, 'colsample_bylevel': 0.7405046763756069, 'colsample_bytree': 0.6151399892026908, 'eta': 0.22398969443165967, 'gamma': 5.745168482871384e-07, 'lambda': 0.019440307789833522, 'max_depth': 10, 'min_child_weight': 0.16853765259109013, 'subsample': 0.6197753137030573, 'objective': 'binary:logistic', 'eval_metric': 'logloss', 'silent': 1}
roc_auc = 0.777906
saved state to results/tracker_test_XGBoost.pickle
~~~~~~~~~~~~~~~~~~~~ LightGBM ~~~~~~~~~~~~~~~~~~~~
saved state to results/tracker_test_LightGBM.pickle
Default LightGBM result on CV:

loss = 0.406009595437
best_n_estimators = 29
params = {'boosting_type': 'gbdt', 'colsample_bytree': 1, 'drop_rate': 0.1, 'is_unbalance': False, 'learning_rate': 0.1, 'max_bin': 2

{'criterion': 'gini', 'max_depth': 8, 'max_features': 4, 'n_estimators': 8, 'normalize': 1, 'scale': 1}
{'max_depth': 1, 'max_features': 4, 'n_estimators': 10, 'criterion': 'gini', 'verbose': 0}
saved state to results/tracker_test_RandomForestClassifier.pickle
Default RandomForestClassifier result on CV:

loss = 0.709934978007
best_n_estimators = 59
params = {'max_depth': 1, 'max_features': 4, 'n_estimators': 10, 'criterion': 'gini', 'verbose': 0}
XYCDataset(X=array([[    0.        ,     0.        ,     0.        , ...,
            0.        ,     0.        ,     0.        ],
       [    0.        ,    41.34658432,   222.88896179, ...,
            0.        ,     0.        ,     0.        ],
       [ 1021.92480469,    13.87764549,   541.56542969, ...,
            0.        ,     0.        ,     0.        ],
       ..., 
       [    0.        ,   309.15771484,   488.13595581, ...,
            0.        ,     0.        ,     0.        ],
       [    0.        ,     0.        ,   197.3000

AttributeError: 'XYCDataset' object has no attribute 'get_label'

## Compare

In [ ]:
metric, mes_min = 'roc_auc', False
full_results = {}
for i in CANDIDATES.keys():
    if i in trackers:
        tracker = trackers[i]
    else:
        tracker = tracker_factory(model_name=i)
        tracker.load_state()
    full_results.update({i:{'tuned': tracker.state['tuned_test'], 'default': tracker.state['default_test']}})

In [ ]:
def plot_metric_results(full_results, index, metric, is_min_better=True):
    test_results_list = []
    for i in index:
        test_results_list.append([full_results[i]['default'][metric], full_results[i]['tuned'][metric]])
        
    test_results = np.array(test_results_list)
    if is_min_better:
        baseline = test_results.min()
    else:
        baseline = test_results.max()
    diff = 100 * test_results / baseline - 100
    test_results_formatted = [['{:.6f} ({:+.2f}%)'.format(test_results[i, j], diff[i, j]) for j in range(2)] for i in range(len(index))]

    print (pd.DataFrame(test_results_formatted, columns=['default', 'tuned'], index=index))
    
    full_names = [" ".join(i) for i in itertools.product(index, ['default', 'tuned'])]

    named_results = zip(full_names, test_results.flatten())

    sorted_results = sorted(named_results, key=lambda x: x[1], reverse=not is_min_better)
    xticks = ['%s\n%.5f' % (name, loss) for name, loss in sorted_results]

    pyplot.figure(figsize=(20, 7))
    pyplot.scatter(range(len(full_names)), list(zip(*sorted_results))[1], s=150)
    pyplot.xticks(range(len(full_names)), xticks, fontsize=15)
    pyplot.yticks(fontsize=12)
    pyplot.title('Comparison', fontsize=20)
    pyplot.ylabel(metric, fontsize=16)

In [ ]:
%pylab inline --no-import-all
metric, is_min_better = 'roc_auc', False
plot_metric_results(full_results, CANDIDATES.keys(), metric, is_min_better=is_min_better)